<a href="https://colab.research.google.com/github/7ZXU/AI/blob/main/HandsOnMachineLearning/HandsOnCh4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**4.1 선형회귀**
**4.1.1 정규방정식**


비용함수 최소화하는 세타값 바로 구하는 공식

In [ ]:
import numpy as np
#선형 데이터 생성
X=2*np.random.rand(100,1)
y=4+3*X+np.random.randn(100,1) 
#############.randn
#가우시안_잡음
#잡음이 추가된 데이터 생성

In [ ]:
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

plt.plot(X,y,"b.")
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0,fontsize=18)
plt.axis([0,2,0,15])
plt.show()

In [ ]:
X_b=np.c_[np.ones((100,1)),X]
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
#정규방정식
#np.linalg #module #선형대수 
#inv() #함수 #역행렬계산
#dot() #method #행렬곱셉

theta_best 
#최적화 theta 계산
#가우시안 잡음 때문에 정확히 재현해내지 못함

#정규방정식이 가장 정확

In [ ]:
#theata 사용 예측

X_new = np.array([[0], [2]])
X_new_b = np.c_[np.ones((2,1)), X_new] 
##############모든 샘플에 x0=1추가
y_predict = X_new_b.dot(theta_best)
y_predict



In [ ]:
plt.plot(X_new, y_predict, "r-", linewidth=2, label="예측")
plt.plot(X, y, "b.")
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.legend(loc="upper left", fontsize=14)
plt.axis([0, 2, 0, 15])
plt.show()

**Scikit-Learn에서 LinearRegression 사용하기**

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X,y)

In [ ]:
lin_reg.intercept_, lin_reg.coef_
#intercept_ #편향
#coef_ #가중치
#편향과 가중치 분리하여 저장

In [ ]:
lin_reg.predict(X_new)

##**4.2 경사하강법**
**4.2.1 경사하강법**

In [ ]:
eta = 0.1 #learning rate
#오차 = 실제값-예측값
#파라미터 공간에 얼마만큼의 영향력을 행사할지 직접 정하는 hyperaprameter
n_iterations = 1000
m=100

weight = np.random.randn(2,1) #random init

for step in range(n_iterations) : 
    gradients = 2/m * X_b.T.dot(X_b.dot(weight) - y)
    weight = weight - eta*gradients 

    if (step+1) % 200 == 0:
        print('Step : {:04d}, weight = \n {}'.format(step+1, weight))


print("최종 결과값 : \n{}".format(weight))

**4.2.2 확률적 경사 하강법**

In [ ]:
weight_path_sgd = []
m=len(X_b)
np.random.seed(42)

In [ ]:
n_epochs=50
t0, t1 = 5, 50 
#학습 스케줄 hyperparameter
#learning rate schedule
#매 반복에서 학습률 결정

def learning_schedule(t):
    return t0 / (t+t1)
weight = np.random.randn(2,1) #random init

for epoch in range(n_epochs): #epoch #반복
    for i in range(m): #m #훈련 샘플 수 
        random_index = np.random.randint(m)
        xi = X_b[random_index : random_index+1]
        yi = y[random_index : random_index+1]
        gradients = 2* xi.T.dot(xi.dot(weight)-yi)
        eta = learning_schedule(epoch*m+i)
        weight = weight - eta*gradients

In [ ]:
weight

In [ ]:
from sklearn.linear_model import SGDRegressor #제곱오차함수 최적화
sgd_reg = SGDRegressor(
    max_iter=50, #반복횟수
    penalty=None, 
    eta0=0.1, #학습률
    random_state=42
    )

sgd_reg.fit(X, y.ravel())


In [ ]:
sgd_reg.intercept_, sgd_reg.coef_

**4.2.3 미니배치 경사 하강법**

매 스텝에서 미니 배치 추출하여 gradient 계산

행렬연산에 최적화된 하드웨어, GPU 에서 빠르게 수행

- 장점 : 덜 불규칙하게 감소 
- 당점 : local minimum 빠질 확률 높음

##**4.3 다항회귀(Polynomial Regression)**

선형 모델로 비선형 데이터 학습
방법 : 특성의 거듭제곱을 새로운 특성으로 추가


In [ ]:
#2차 방정식 #비선형 데이터셋
m=100
X=6*np.random.rand(m,1)-3
y=0.5*X**2+X+2+np.random.randn(m, 1)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=2, include_bias=False)
#제곱함수

X_poly = poly_features.fit_transform(X)
print('X[0]:', X[0])
print('X_poly[0]:', X_poly[0])
#[특성, 특성**2]

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y) #훈련

In [ ]:
lin_reg.intercept_, lin_reg.coef_

##**4.4 학습곡선(Learning Curve)**

고차 다항회귀는 선형회귀보다 train set에 overfitting될 위험이 크다 

따라서 과적합 확인하는 방법 -> 학습곡선

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def plot_learning_curves(model, X, y):
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=10 
                                                      )
    train_errors, val_errors = [], []
    
    for m in range(1, len(X_train)):
        model.fit(X_train[:m],y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        train_errors.append( #훈련오차
            mean_squared_error(
                y_train[:m], y_train_predict))
        val_errors.append( #검증오차
            mean_squared_error(
                y_val, y_val_predict)
            )
        
        plt.plot(np.sqrt(train_errors),"r-+", linewidth=2,label="훈련")
        plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="검증")
        plt.legend(loc="upper right", fontsize=14)
        plt.xlabel("훈련세트크기",fontsize=14)
        plt.ylabel("RMSE", fontsize=14)

    

In [ ]:
lin_reg = LinearRegression()
plot_learning_curves(lin_reg, X, y)
plt.axis([0,80,0,3])
plt.show()

#과소적합모델 해결방법
#1 더 복잡한 모델 사용
#2 더 나은 특성 사용

In [ ]:
from sklearn.pipeline import Pipeline
polynominal_regression = Pipeline([
        ("poly_features", PolynomialFeatures(
            degree=10, include_bias=False
        )), #10차 다항회귀모델
        ("lin_reg", LinearRegression()),
])

plot_learning_curves(polynominal_regression, X, y)
plt.axis([0,80,0,3])
plt.show()


#과대적합 모델 
#훈련데이터 오차는 선형회귀보다 낮다 

#과대적합 해결방법
#더 많은 훈련 데이터 추가 

##**4.5 규제(Regularization)가 있는 선형 모델**
**4.5.1 릿지회귀(Ridge Regression)**

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

np.random.seed(42)
m = 20
X = 3 * np.random.rand(m, 1)
y = 1 + 0.5 * X + np.random.randn(m, 1) / 1.5
X_new = np.linspace(0, 3, 100).reshape(100, 1)

def plot_model(model_class, polynomial, alphas, **model_kargs):
    for alpha, style in zip(alphas, ("b-", "g--", "r:")):
        model = model_class(alpha, **model_kargs) if alpha > 0 else LinearRegression()
        if polynomial:
            model = Pipeline([
                    ("poly_features", PolynomialFeatures(degree=10, include_bias=False)),
                    ("std_scaler", StandardScaler()),
                    ("regul_reg", model),
                ])
        model.fit(X, y)
        y_new_regul = model.predict(X_new)
        lw = 2 if alpha > 0 else 1
        plt.plot(X_new, y_new_regul, style, linewidth=lw, label=r"$\alpha = {}$".format(alpha))
    plt.plot(X, y, "b.", linewidth=3)
    plt.legend(loc="upper left", fontsize=15)
    plt.xlabel("$x_1$", fontsize=18)
    plt.axis([0, 3, 0, 4])

plt.figure(figsize=(10,4))
plt.subplot(121)
plot_model(Ridge, polynomial=False, alphas=(0, 10, 100), random_state=42)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.subplot(122)
plot_model(Ridge, polynomial=True, alphas=(0, 10**-5, 1), random_state=42)

plt.show()

In [ ]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha=1, solver='cholesky', random_state=42)  # alpha == lambda
ridge_reg.fit(X, y)

In [ ]:
ridge_reg.predict([[1.5]])

**4.5.2 라쏘 회귀**

In [ ]:
from sklearn.linear_model import Lasso

plt.figure(figsize=(10,4))
plt.subplot(121)
plot_model(Lasso, polynomial=False, alphas=(0, 0.1, 1), random_state=42)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.subplot(122)
plot_model(Lasso, polynomial=True, alphas=(0, 10**-7, 1), tol=1, random_state=42)
plt.show()

In [ ]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(X, y)

In [ ]:
lasso_reg.predict([[1.5]])

In [ ]:
sgd_reg = SGDRegressor(max_iter=5, penalty="l1", random_state=42)
sgd_reg.fit(X, y.ravel())
sgd_reg.predict([[1.5]])

**4.5.3 ElasticNet**

In [ ]:
from sklearn.linear_model import ElasticNet

elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
elastic_net.fit(X, y)
elastic_net.predict([[1.5]])

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(121)
plot_model(ElasticNet, polynomial=False, alphas=(0, 0.1, 1), random_state=42)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.subplot(122)
plot_model(ElasticNet, polynomial=True, alphas=(0, 10**-7, 1), tol=1, random_state=42)
plt.show()

**4.5.4 조기 종료**

최솟값에 도달하면 학습 스텝이 남았더라도 학습 중지



In [ ]:
np.random.seed(42)
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 2 + X + 0.5 * X**2 + np.random.randn(m, 1)

X_train, X_val, y_train, y_val = train_test_split(X[:50], y[:50].ravel(), test_size=0.5, random_state=10)

poly_scaler = Pipeline([
        ("poly_features", PolynomialFeatures(degree=90, include_bias=False)),
        ("std_scaler", StandardScaler()),
    ])

X_train_poly_scaled = poly_scaler.fit_transform(X_train)
X_val_poly_scaled = poly_scaler.transform(X_val)

sgd_reg = SGDRegressor(max_iter=1,
                       penalty=None,
                       eta0=0.0005,
                       warm_start=True,
                       learning_rate="constant",
                       random_state=42)

n_epochs = 500
train_errors, val_errors = [], []
for epoch in range(n_epochs):
    sgd_reg.fit(X_train_poly_scaled, y_train)
    y_train_predict = sgd_reg.predict(X_train_poly_scaled)
    y_val_predict = sgd_reg.predict(X_val_poly_scaled)
    train_errors.append(mean_squared_error(y_train, y_train_predict))
    val_errors.append(mean_squared_error(y_val, y_val_predict))

best_epoch = np.argmin(val_errors)
best_val_rmse = np.sqrt(val_errors[best_epoch])

plt.annotate('최선의 모델',
             xy=(best_epoch, best_val_rmse),
             xytext=(best_epoch, best_val_rmse + 1),
             ha="center",
             arrowprops=dict(facecolor='black', shrink=0.05),
             fontsize=16,
            )

best_val_rmse -= 0.03  # just to make the graph look better
plt.plot([0, n_epochs], [best_val_rmse, best_val_rmse], "k:", linewidth=2)
plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="검증 세트")
plt.plot(np.sqrt(train_errors), "r--", linewidth=2, label="훈련 세트")
plt.legend(loc="upper right", fontsize=14)
plt.xlabel("에포크", fontsize=14)
plt.ylabel("RMSE", fontsize=14)
plt.show()

##**4.6 로지스틱회귀**

이진 분류 알고리즘 중 하나

**4.6.1 확률 추정**

In [ ]:
t = np.linspace(-10, 10, 100)
sig = 1 / (1 + np.exp(-t))

plt.figure(figsize=(9, 3))
plt.plot([-10, 10], [0, 0], "k-")
plt.plot([-10, 10], [0.5, 0.5], "k:")
plt.plot([-10, 10], [1, 1], "k:")
plt.plot([0, 0], [-1.1, 1.1], "k-")
plt.plot(t, sig, "b-", linewidth=2, label=r"$\sigma(t) = \frac{1}{1 + e^{-t}}$")
plt.xlabel("t")
plt.legend(loc="upper left", fontsize=20)
plt.axis([-10, 10, -0.1, 1.1])
plt.show()

**4.6.2 훈련과 비용 함수**

**4.6.3 결정 경계**

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
list(iris.keys())

In [ ]:
print(iris.DESCR)

In [ ]:
import pandas as pd

In [ ]:
iris_df = pd.DataFrame(iris['data'], columns=iris.feature_names)
iris_df.head()

In [ ]:
iris_df.describe()

**Pental width 이용한 Verginca종 분류기 구현**

꽃잎 너비 이용해서 종 분류

In [ ]:
X = iris['data'][:, 3:]  # 꽃잎 너비
y = (iris['target'] == 2).astype(np.int)  # Verginca == 1

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(random_state=42)
log_reg.fit(X, y)

In [ ]:
X_new = np.linspace(0, 3, 1000).reshape(-1, 1)
y_proba = log_reg.predict_proba(X_new)
decision_boundary = X_new[y_proba[:, 1] >= 0.5][0]

plt.figure(figsize=(8, 3))
plt.plot(X[y==0], y[y==0], "bs")
plt.plot(X[y==1], y[y==1], "g^")
plt.plot([decision_boundary, decision_boundary], [-1, 2], "k:", linewidth=2)
plt.plot(X_new, y_proba[:, 1], "g-", linewidth=2, label="Iris-Virginica")
plt.plot(X_new, y_proba[:, 0], "b--", linewidth=2, label="Not Iris-Virginica")
plt.text(decision_boundary+0.02, 0.15, "결정 경계", fontsize=14, color="k", ha="center")
plt.arrow(decision_boundary, 0.08, -0.3, 0, head_width=0.05, head_length=0.1, fc='b', ec='b')
plt.arrow(decision_boundary, 0.92, 0.3, 0, head_width=0.05, head_length=0.1, fc='g', ec='g')
plt.xlabel("꽃잎의 폭 (cm)", fontsize=14)
plt.ylabel("확률", fontsize=14)
plt.legend(loc="center left", fontsize=14)
plt.axis([0, 3, -0.02, 1.02])
plt.show()

In [ ]:
decision_boundary

In [ ]:
log_reg.predict([[1.7], [1.5]])

**Petal width, Petal length 이용한 종 분류기**

In [ ]:
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = (iris["target"] == 2).astype(np.int)

log_reg = LogisticRegression(C=10**10, random_state=42)
log_reg.fit(X, y)

x0, x1 = np.meshgrid(
        np.linspace(2.9, 7, 500).reshape(-1, 1),
        np.linspace(0.8, 2.7, 200).reshape(-1, 1),
    )
X_new = np.c_[x0.ravel(), x1.ravel()]

y_proba = log_reg.predict_proba(X_new)

plt.figure(figsize=(10, 4))
plt.plot(X[y==0, 0], X[y==0, 1], "bs")
plt.plot(X[y==1, 0], X[y==1, 1], "g^")

zz = y_proba[:, 1].reshape(x0.shape)
contour = plt.contour(x0, x1, zz, cmap=plt.cm.brg)


left_right = np.array([2.9, 7])
boundary = -(log_reg.coef_[0][0] * left_right + log_reg.intercept_[0]) / log_reg.coef_[0][1]

plt.clabel(contour, inline=1, fontsize=12)
plt.plot(left_right, boundary, "k--", linewidth=3)
plt.text(3.5, 1.5, "Iris-Virginica 아님", fontsize=14, color="b", ha="center")
plt.text(6.5, 2.3, "Iris-Virginica", fontsize=14, color="g", ha="center")
plt.xlabel("꽃잎의 길이", fontsize=14)
plt.ylabel("꽃잎의 폭", fontsize=14)
plt.axis([2.9, 7, 0.8, 2.7])
plt.show()

**4.6.4 소프트맥스 회귀(Softmax Regression)**

**Scikit-Learn LogisticRegression 이용하여 Softmax Regression 사용**

In [ ]:
X = iris["data"][:, (2, 3)]  # 꽃잎 길이, 꽃잎 너비
y = iris["target"]

softmax_reg = LogisticRegression(multi_class="multinomial",solver="lbfgs", C=10, random_state=42)
softmax_reg.fit(X, y)

In [ ]:
x0, x1 = np.meshgrid(
        np.linspace(0, 8, 500).reshape(-1, 1),
        np.linspace(0, 3.5, 200).reshape(-1, 1),
    )
X_new = np.c_[x0.ravel(), x1.ravel()]


y_proba = softmax_reg.predict_proba(X_new)
y_predict = softmax_reg.predict(X_new)

zz1 = y_proba[:, 1].reshape(x0.shape)
zz = y_predict.reshape(x0.shape)

plt.figure(figsize=(10, 4))
plt.plot(X[y==2, 0], X[y==2, 1], "g^", label="Iris-Virginica")
plt.plot(X[y==1, 0], X[y==1, 1], "bs", label="Iris-Versicolor")
plt.plot(X[y==0, 0], X[y==0, 1], "yo", label="Iris-Setosa")

from matplotlib.colors import ListedColormap
custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])

plt.contourf(x0, x1, zz, cmap=custom_cmap)
contour = plt.contour(x0, x1, zz1, cmap=plt.cm.brg)
plt.clabel(contour, inline=1, fontsize=12)
plt.xlabel("꽃잎의 길이", fontsize=14)
plt.ylabel("꽃잎의 폭", fontsize=14)
plt.legend(loc="center left", fontsize=14)
plt.axis([0, 7, 0, 3.5])
plt.show()

In [ ]:
softmax_reg.predict([[5, 2]])

In [ ]:
softmax_reg.predict_proba([[5, 2]])